In [ ]:
# !pip install -r ./requirements.txt

In [ ]:
# !pip uninstall virny -y

In [ ]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [ ]:
# !pip install lightgbm==3.3.5

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [ ]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

## Import dependencies

In [ ]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSEmploymentDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_preprocessing_intervention

## Define Input Variables

In [ ]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
# FAIR_INTERVENTION_PARAMS_LST = [0.0 + 0.1 * i for i in range(11)]
FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5,
                                0.6000000000000001, 0.7, 0.8, 0.9, 1.0]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'folk_ny_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

In [ ]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [ ]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

## Initialize custom objects

In [ ]:
data_loader = ACSEmploymentDataset(state=['NY'], year=2018, with_nulls=False,
                                   subsample_size=20_000, subsample_seed=42)
data_loader.X_data.head()

In [ ]:
data_loader.X_data.shape

## Run experiment iterations

### Experiment iteration 1

In [ ]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             with_tuning=True,
#                                              with_tuning=False,
#                                              tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 2

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 7

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 7
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 8

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 8
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 9

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 9
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 10

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 10
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)